MASTER PYTHON - Using the full training set - Running the learning models take 45 minute each, be patient

I added a line from FYANGO that take sthe vectorizer to the app that builds teh external question faster

All the models are exported

2 models, LOG and AggrO

Modify the exisitng models reliably predict if new
s is fake or real, using exisitng KAGGEL datasets

#### Dataset used - https://www.kaggle.com/fake-news/data

### Dataset Description

train.csv: A full training dataset with the following attributes:

* id: unique id for a news article
* title: the title of a news article
* author: author of the news article
* text: the text of the article; could be incomplete
* label: a label that marks the article as potentially unreliable
  * 1: FAKE
  * 0: TRUE


Ensure the file creator is installed to save the MODELS for use in Heroku app

Set the dependencies

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/jonowood/Project-4-A-Team/blob/JonBranch/ML/JONO_PREDICT_MASTER.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a href="https://github.com/jonowood/Project-4-A-Team/blob/JonBranch/ML/JONO_PREDICT_MASTER.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import re 
import nltk 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
import itertools

import pickle


The Natural Language Toolkit (NLTK) is a platform used for building Python programs that work with human language data for applying in statistical natural language processing (NLP). It contains text processing libraries for tokenization, parsing, classification, stemming, tagging and semantic reasoning.
The nltk.corpus package defines a collection of corpus reader classes, which can be used to access the contents of a diverse set of corpora. The list of available corpora is given at: https://www.nltk.org/nltk_data/ Each corpus reader class is specialized to handle a specific corpus forma

Load and test all the STW ,Stopwords are words which occur frequently in a corpus. e.g a, an, the, in. Frequently occurring words are removed from the corpus for the purpose of text-normalization.

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jonow\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# check if Stopwords laoded in english

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing and Analysis


Regular Expression Syntax. A regular expression (or RE) specifies a set of strings that matches it; the functions in this module let you check if a particular string matches a given regular expression (or if a given regular expression matches a particular string, which comes down to the same thing

Load the dataset, Read

In [4]:
# First let's load the dataset
news_dataset = pd.read_csv('../Dataset/train.csv',encoding = "latin-1")


In [5]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1


In [6]:
news_dataset.shape

(20800, 5)

In [7]:
# Remove all columns between column index 5 to 250
news_dataset.drop(news_dataset.iloc[:, 5:251], inplace=True, axis=1)
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1


In [8]:
# Checking for missing values
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
# Let's replace the missing values with empty strings wherever possible
news_dataset = news_dataset.fillna('')

In [10]:
news_dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

:In this dataset the Title, Author indicates validity of the message, various previous works validates teh statement that authnetic authors makes a difference and newsrooms use thsi regulary as key indicators to tack their messages

In [11]:
news_dataset['text_corpus'] = news_dataset['author']+' '+news_dataset['title']+' '+news_dataset['text']

In [12]:
print(news_dataset['text_corpus'])

0        Darrell Lucus House Dem Aide: We Didnât Even...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a âPoster C...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799    David Swanson What Keeps the F-35 Alive   Davi...
Name: text_corpus, Length: 20800, dtype: object


In [13]:
news_dataset.head()

,id,title,author,text,label,text_corpus
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1,Darrell Lucus House Dem Aide: We Didnât Even...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1,Howard Portnoy Iranian woman jailed for fictio...


In [14]:
# Now we will separate the data and label i.e. text_corpus and label fields
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [15]:
X = news_dataset['text_corpus']
print(X)

0        Darrell Lucus House Dem Aide: We Didnât Even...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a âPoster C...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799    David Swanson What Keeps the F-35 Alive   Davi...
Name: text_corpus, Length: 20800, dtype: object


In [16]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


In [17]:
X.head()

0    Darrell Lucus House Dem Aide: We Didnât Even...
1    Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2    Consortiumnews.com Why the Truth Might Get You...
3    Jessica Purkiss 15 Civilians Killed In Single ...
4    Howard Portnoy Iranian woman jailed for fictio...
Name: text_corpus, dtype: object




Stemming or Lemmatization-https://pythonprogramming.net/stemming-nltk-tutorial/

Stemming is a technique used to reduce an inflected word down to its word stem. For example, the words “programming,” “programmer,” and “programs” can all be reduced down to the common word stem “program.” In other words, “program” can be used as a synonym for the prior three inflection words

Porter’s Stemmer applies a set of five sequential rules (also called phases) to determine common suffixes from sentences.


In [18]:
port_stem = PorterStemmer()

def stemming(content):
    # Pick all alphabet characters - lowercase and uppercase...all others such as numbers and punctuations will be removed. Numbers or punctuations will be replaced by a whitespace
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    # Converting all letters to lowercase 
    stemmed_content = stemmed_content.lower()
    # Converting all to a splitted case or a list
    stemmed_content = stemmed_content.split()
    # Applying stemming, so we get the root words wherever possible + remove stopwords as well
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    # Join all the words in final content
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [19]:
news_dataset['text_corpus'] = news_dataset['text_corpus'].apply(stemming)

## Getting a coffee, let me know when its done ☕

In [20]:
import winsound
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

In [21]:
print(news_dataset['text_corpus'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2        consortiumnew com truth might get fire truth m...
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799    david swanson keep f aliv david swanson author...
Name: text_corpus, Length: 20800, dtype: object


In [22]:
# Separating data and label
X = news_dataset['text_corpus'].values
Y = news_dataset['label'].values

In [23]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet hous dem aid even see comey letter jason chaffetz tweet darrel lucu octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common licens apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehous octob cours know case comey actual say review email light unrel 

In [24]:
print(Y)

[1 0 1 ... 0 1 1]


TF-IDF (Term Frequency, Inverse Document Frequency)

### Converting Textual data to Numerical data

* The TF-IDF Vectorizer
* TF-IDF Vectorizer coverts textual data to numerical data

Thsi is still a bit messed up and need to be cleaned, I stidued the HEROKYU app fiel and the vectorizer is use dto translate the input text to ML to do teh comparison

In [25]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

TfidfVectorizer()

In [26]:
X = vectorizer.transform(X)

In [27]:
pickle.dump(vectorizer, open('../Pickles/tfidfvect2.pkl', 'wb'))

In [28]:
TEST_model = pickle.load(open('../Pickles/tfidfvect2.pkl', 'rb'))

print(TEST_model)

TfidfVectorizer()


In [29]:
print(X)

  (0, 109752)	0.049158312425168854
  (0, 109697)	0.0190646711515277
  (0, 108742)	0.04416544119908134
  (0, 108738)	0.09477494042884232
  (0, 108695)	0.03758488097939004
  (0, 108658)	0.01130614774071694
  (0, 108007)	0.017092546683505856
  (0, 107190)	0.017105936674103112
  (0, 107099)	0.012543234221230963
  (0, 107013)	0.029126417104928328
  (0, 106934)	0.012863319680563097
  (0, 106734)	0.011771716334271506
  (0, 105884)	0.025727197929110487
  (0, 105848)	0.031296701378124764
  (0, 104837)	0.02153649554212262
  (0, 103422)	0.06544555398259812
  (0, 102736)	0.03314918847150756
  (0, 102485)	0.01639612818098454
  (0, 101717)	0.038071924979380216
  (0, 101077)	0.011082403436475742
  (0, 101067)	0.0432044670628921
  (0, 101014)	0.13602128375819167
  (0, 100866)	0.0713092337063475
  (0, 99577)	0.03944988916619374
  (0, 99009)	0.027120358929731154
  :	:
  (20799, 7470)	0.010635431711878486
  (20799, 7143)	0.02816704434978389
  (20799, 6848)	0.03959171777516513
  (20799, 6810)	0.0253655855

Modeling & Model Evaluation

### Splitting the data into test and train datasets

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.18, random_state=42)

We use 2 models to determine the accuracy of teh training set and will then select the most accurate model to us ein HEREKO
The first Model - Logistic regression

In [31]:
# Training the model
logisticreg_model = LogisticRegression()

logisticreg_model.fit(X_train, Y_train)

LogisticRegression()

In [ ]:
# history_dict = history.history
# history_dict.keys()

### Model Evaluation

In [32]:
# Accuracy Score on Training Data
X_train_prediction = logisticreg_model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

print('Accuracy score on the training data: ',training_data_accuracy)

# Accuracy Score on Test Data
X_test_prediction = logisticreg_model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

print('Accuracy score on the test data: ',test_data_accuracy)

Accuracy score on the training data:  0.9790103189493433
Accuracy score on the test data:  0.9564636752136753


In [33]:
import pickle
pickle.dump(logisticreg_model, open('../Pickles/logisticreg_model.pkl', 'wb'))


In [34]:
# Classification report for test data
classification_report(Y_test, X_test_prediction)

'              precision    recall  f1-score   support\n\n           0       0.96      0.95      0.96      1919\n           1       0.95      0.96      0.96      1825\n\n    accuracy                           0.96      3744\n   macro avg       0.96      0.96      0.96      3744\nweighted avg       0.96      0.96      0.96      3744\n'

**CLASSIFICATION MODEL : PASSIVE AGGRESSIVE CLASSIFIER**

* Passive Aggressive Classifier works by responding as passive for correct classifications and responding as aggressive for any miscalculation.

In [35]:
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [36]:
# Importing modules
# from sklearn.datasets import load_iris
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

# Splitting dataset into train and test sets
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X, Y, test_size=0.33, random_state=42)

# Creating model
passiveagressive_model = PassiveAggressiveClassifier(C = 0.5, random_state = 5)

# Fitting model
passiveagressive_model.fit(X2_train, Y2_train)

# Making prediction on test set
test_pred = passiveagressive_model.predict(X2_test)

# Model evaluation
print(f"Test Set Accuracy : {accuracy_score(Y2_test, test_pred) * 100} %\n\n")

#print(f"Classification Report : \n\n{classification_report(Y2_test, test_pred)}")


Test Set Accuracy : 96.70745920745921 %




In [37]:
pickle.dump(passiveagressive_model, open('../Pickles/passiveagressive_model.pkl', 'wb'))

Testing the two models

In [67]:
y_pred = logisticreg_model.predict(X_test)

# Calculate the prediction accuracy
accuracy = np.mean(y_pred == Y_test) * 100

# Print the accuracy
print("Prediction accuracy: {:.2f}%".format(accuracy))

# Print the prediction for a single example
X_new = X_test[501]
prediction = logisticreg_model.predict(X_new.reshape(1, -1))
print("Prediction for example 500: ", prediction[0])
if (prediction[0] == 0):
  print('Jono says its True')
else:
  print('Johan Says it is a porky:)')

Prediction accuracy: 95.65%
Prediction for example 500:  1
Johan Says it is a porky:)


In [68]:
y2_pred = logisticreg_model.predict(X2_test)

# Calculate the prediction accuracy
accuracy = np.mean(y2_pred == Y2_test) * 100

# Print the accuracy
print("Prediction accuracy: {:.2f}%".format(accuracy))

# Print the prediction for a single example
X2_new = X2_test[501]
prediction2 = prediction2 = passiveagressive_model.predict(X2_new.reshape(1, -1))
print("Prediction for example 500: ", prediction[0])
if (prediction[0] == 0):
  print('Jono says its True')
else:
  print('Johan Says it is a porky:)')

Prediction accuracy: 96.46%
Prediction for example 500:  1
Johan Says it is a porky:)


In [66]:
news_dataset[100:101]

,id,title,author,text,label,text_corpus
100,100,Technocracy: The Real Reason Why The UN Wants ...,Activist Post,"By Patrick Wood By its very nature, the Intern...",1,activist post technocraci real reason un want ...


In [41]:
print(Y_test[100])

1


In [42]:
X_new = X_test[301]

prediction = logisticreg_model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Jono says its True')
else:
  print('Johan Says it is a porky:)')

[1]
Johan Says it is a porky:)


In [43]:
print(Y_test[301])

1


In [44]:
news_dataset[300:301]

,id,title,author,text,label,text_corpus
300,300,M.T.A. Shortens L Train Shutdown to 15 Months ...,Emma G. Fitzsimmons,The Metropolitan Transportation Authority had ...,0,emma g fitzsimmon shorten l train shutdown mon...


In [50]:
vectorizer.fit(X)

AttributeError: lower not found

In [49]:
pickled_model1 = pickle.load(open('logisticreg_model.pkl', 'rb'))
pickled_model1.predict(X)

AttributeError: lower not found

In [ ]:
pickled_model2 = pickle.load(open('passiveagressive_model.pkl', 'rb'))
pickled_model2.predict(X2_test)

FAngo tested a point to clarify teh vector model

In [ ]:
ps = PorterStemmer()

In [ ]:
review = re.sub('[^a-zA-Z]', ' ', news_dataset['text_corpus'][100])
review = review.lower()
review = review.split()
    
review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
review = ' '.join(review)
review

In [ ]:
val = vectorizer.transform([review]).toarray()

In [ ]:
tfidfvect2_model2 = pickle.load(open('tfidfvect2.pkl', 'rb'))
